In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder

In [2]:
df = pd.read_csv('data/20220208_Final_result.csv')

# Data exploration

### Remove Null Price rows and change type to float

In [3]:
df = df[pd.to_numeric(df['Price'], errors='coerce').notnull()]

In [4]:
df = df.astype({"Price": float}, errors='raise')

### Remove duplicate ID

In [5]:
df['Immoweb ID'].value_counts()

9691876    4
9662650    3
9597143    3
8237073    3
9687272    3
          ..
9623733    1
9245768    1
9606238    1
9695276    1
7921699    1
Name: Immoweb ID, Length: 13893, dtype: int64

In [6]:
df = df.drop_duplicates(['Immoweb ID'], keep='last')
df['Immoweb ID'].value_counts()

9729720    1
9678715    1
9697071    1
9557609    1
9678634    1
          ..
9518030    1
9622950    1
9478102    1
9620790    1
7921699    1
Name: Immoweb ID, Length: 13893, dtype: int64

### Remove subtype 

In [7]:
df = df[df['property sub-type'] != "APARTMENT_BLOCK"]
df['property sub-type'].value_counts()

VILLA                   2468
GROUND_FLOOR            1606
DUPLEX                  1580
MIXED_USE_BUILDING      1240
PENTHOUSE               1170
FLAT_STUDIO              873
EXCEPTIONAL_PROPERTY     566
SERVICE_FLAT             500
MANSION                  479
TOWN_HOUSE               383
COUNTRY_COTTAGE          292
LOFT                     271
BUNGALOW                 212
FARMHOUSE                171
TRIPLEX                   99
KOT                       95
CHALET                    94
MANOR_HOUSE               62
CASTLE                    50
Name: property sub-type, dtype: int64

# Data Cleaning

### Building condition

In [8]:
df['Building condition'].value_counts()
df['Building condition'].unique()

array(['Good', nan, 'As new', 'To renovate', 'To be done up',
       'Just renovated', 'To restore'], dtype=object)

In [9]:
building_condition_map = {'As new': 6, 'Just renovated': 5, 'Good': 4, 'To be done up': 3, 'To renovate':2, 'To restore':1}
df = df.applymap(lambda s: building_condition_map.get(s) if s in building_condition_map else s)

In [10]:
df['Building condition'].isnull().sum()

3828

In [11]:
df['Building condition'] = df['Building condition'].fillna(2)
df['Building condition'].isnull().sum()

0

### Property type

In [12]:
df = pd.get_dummies(df, columns=['Property type', 'Post code'])

### Kitchen type	


In [13]:
Kit_type_dict = {"USA uninstalled" : 0, 
                 "Not installed" : 0, 
                 "Installed": 1, 
                 "USA installed": 1,
                 "Semi equipped": 1,
                 "USA semi equipped": 1,
                 "Hyper equipped": 2,
                 "USA hyper equipped": 2,
                 "Nan" : 0
                 
                }

df = df.replace(Kit_type_dict)
df["Kitchen type"] = df["Kitchen type"].fillna(0)


df

,Immoweb ID,property sub-type,Price,Building condition,Kitchen type,Bedrooms,Furnished,Terrace surface,Tenement building,Number of frontages,...,Post code_9960,Post code_9968,Post code_9970,Post code_9971,Post code_9981,Post code_9982,Post code_9988,Post code_9990,Post code_9991,Post code_9992
0,9729720,PENTHOUSE,179000.0,4.0,1.0,1.0,Yes,9.0,No,NaN,...,0,0,0,0,0,0,0,0,0,0
1,9729785,PENTHOUSE,255000.0,4.0,1.0,1.0,Yes,46.0,No,2.0,...,0,0,0,0,0,0,0,0,0,0
2,9729784,PENTHOUSE,255000.0,4.0,1.0,1.0,Yes,46.0,No,2.0,...,0,0,0,0,0,0,0,0,0,0
3,9729780,PENTHOUSE,620000.0,4.0,1.0,3.0,Yes,70.0,No,2.0,...,0,0,0,0,0,0,0,0,0,0
4,9727201,PENTHOUSE,379000.0,4.0,2.0,2.0,No,110.0,No,3.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14536,7770552,SERVICE_FLAT,132000.0,6.0,0.0,1.0,NaN,NaN,No,NaN,...,0,0,0,0,0,0,0,0,0,0
14537,7770553,SERVICE_FLAT,177000.0,6.0,0.0,1.0,NaN,NaN,No,NaN,...,0,0,0,0,0,0,0,0,0,0
14538,7770551,SERVICE_FLAT,190000.0,6.0,0.0,2.0,NaN,NaN,No,NaN,...,0,0,0,0,0,0,0,0,0,0
14539,7921693,SERVICE_FLAT,1350000.0,2.0,0.0,30.0,NaN,NaN,No,NaN,...,0,0,0,0,0,0,0,0,0,0


### Bedrooms


### Furnished


In [14]:
df['Furnished'].unique()

array(['Yes', 'No', nan], dtype=object)

In [15]:
df['Furnished'].isnull().sum()

4080

#### fill No to missing value then map to 1,0

In [16]:
df['Furnished'] = df['Furnished'].fillna("No")
df['Furnished'] = df['Furnished'].apply(lambda v: 0 if v == "No" else 1)

#### double check if value is 1, 0 and no missing value

In [17]:
print(df['Furnished'].unique())
print(df['Furnished'].isna().sum())

[1 0]
0


### Terrace surface + Terrace	


#### Combine terrace & terrace surface in yes and no

In [18]:
df['Terrace_Combined'] = np.where(df['Terrace surface'].isnull() & df['Terrace'].isnull(), 'No', 'Yes')

#### Transform Terrace_Combined to 0 & 1

In [19]:
terrace = ['No', 'Yes']

ordenc = OrdinalEncoder(categories=[terrace])

df[["Terrace_Combined"]] = ordenc.fit_transform(df[["Terrace_Combined"]])

df

,Immoweb ID,property sub-type,Price,Building condition,Kitchen type,Bedrooms,Furnished,Terrace surface,Tenement building,Number of frontages,...,Post code_9968,Post code_9970,Post code_9971,Post code_9981,Post code_9982,Post code_9988,Post code_9990,Post code_9991,Post code_9992,Terrace_Combined
0,9729720,PENTHOUSE,179000.0,4.0,1.0,1.0,1,9.0,No,NaN,...,0,0,0,0,0,0,0,0,0,1.0
1,9729785,PENTHOUSE,255000.0,4.0,1.0,1.0,1,46.0,No,2.0,...,0,0,0,0,0,0,0,0,0,1.0
2,9729784,PENTHOUSE,255000.0,4.0,1.0,1.0,1,46.0,No,2.0,...,0,0,0,0,0,0,0,0,0,1.0
3,9729780,PENTHOUSE,620000.0,4.0,1.0,3.0,1,70.0,No,2.0,...,0,0,0,0,0,0,0,0,0,1.0
4,9727201,PENTHOUSE,379000.0,4.0,2.0,2.0,0,110.0,No,3.0,...,0,0,0,0,0,0,0,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14536,7770552,SERVICE_FLAT,132000.0,6.0,0.0,1.0,0,NaN,No,NaN,...,0,0,0,0,0,0,0,0,0,0.0
14537,7770553,SERVICE_FLAT,177000.0,6.0,0.0,1.0,0,NaN,No,NaN,...,0,0,0,0,0,0,0,0,0,1.0
14538,7770551,SERVICE_FLAT,190000.0,6.0,0.0,2.0,0,NaN,No,NaN,...,0,0,0,0,0,0,0,0,0,1.0
14539,7921693,SERVICE_FLAT,1350000.0,2.0,0.0,30.0,0,NaN,No,NaN,...,0,0,0,0,0,0,0,0,0,0.0


#### Drop Terrace and Terrace Surface columns

In [20]:
df = df.drop(columns =['Terrace', 'Terrace surface'])

df

,Immoweb ID,property sub-type,Price,Building condition,Kitchen type,Bedrooms,Furnished,Tenement building,Number of frontages,Swimming pool,...,Post code_9968,Post code_9970,Post code_9971,Post code_9981,Post code_9982,Post code_9988,Post code_9990,Post code_9991,Post code_9992,Terrace_Combined
0,9729720,PENTHOUSE,179000.0,4.0,1.0,1.0,1,No,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
1,9729785,PENTHOUSE,255000.0,4.0,1.0,1.0,1,No,2.0,NaN,...,0,0,0,0,0,0,0,0,0,1.0
2,9729784,PENTHOUSE,255000.0,4.0,1.0,1.0,1,No,2.0,NaN,...,0,0,0,0,0,0,0,0,0,1.0
3,9729780,PENTHOUSE,620000.0,4.0,1.0,3.0,1,No,2.0,NaN,...,0,0,0,0,0,0,0,0,0,1.0
4,9727201,PENTHOUSE,379000.0,4.0,2.0,2.0,0,No,3.0,No,...,0,0,0,0,0,0,0,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14536,7770552,SERVICE_FLAT,132000.0,6.0,0.0,1.0,0,No,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
14537,7770553,SERVICE_FLAT,177000.0,6.0,0.0,1.0,0,No,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
14538,7770551,SERVICE_FLAT,190000.0,6.0,0.0,2.0,0,No,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
14539,7921693,SERVICE_FLAT,1350000.0,2.0,0.0,30.0,0,No,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0


### Tenement building	


### Swimming pool	


### Garden + Garden surface	


In [21]:
print(df['Garden'].isna().sum())
print(df['Garden'].unique())

11051
[nan 'Yes']


In [22]:
df.loc[df['Garden surface'].isna()]

,Immoweb ID,property sub-type,Price,Building condition,Kitchen type,Bedrooms,Furnished,Tenement building,Number of frontages,Swimming pool,...,Post code_9968,Post code_9970,Post code_9971,Post code_9981,Post code_9982,Post code_9988,Post code_9990,Post code_9991,Post code_9992,Terrace_Combined
0,9729720,PENTHOUSE,179000.0,4.0,1.0,1.0,1,No,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
1,9729785,PENTHOUSE,255000.0,4.0,1.0,1.0,1,No,2.0,NaN,...,0,0,0,0,0,0,0,0,0,1.0
2,9729784,PENTHOUSE,255000.0,4.0,1.0,1.0,1,No,2.0,NaN,...,0,0,0,0,0,0,0,0,0,1.0
3,9729780,PENTHOUSE,620000.0,4.0,1.0,3.0,1,No,2.0,NaN,...,0,0,0,0,0,0,0,0,0,1.0
4,9727201,PENTHOUSE,379000.0,4.0,2.0,2.0,0,No,3.0,No,...,0,0,0,0,0,0,0,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14536,7770552,SERVICE_FLAT,132000.0,6.0,0.0,1.0,0,No,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
14537,7770553,SERVICE_FLAT,177000.0,6.0,0.0,1.0,0,No,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
14538,7770551,SERVICE_FLAT,190000.0,6.0,0.0,2.0,0,No,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
14539,7921693,SERVICE_FLAT,1350000.0,2.0,0.0,30.0,0,No,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0


#### Combine ['Garden'] and ['Garden surface] as ['garden_label] and map to 1,0

In [23]:
def categorise(col):  
    if col['Garden'] == "Yes" or col['Garden surface'] > 0:
        return 1
    return 0


df['garden_label'] = df.apply(lambda col: categorise(col), axis=1)

#### double check if value is 1, 0 and no missing value

In [24]:
df['garden_label'].unique()

array([0, 1])

#### drop column Garden and Garden surface

In [25]:
df = df.drop(columns =['Garden', 'Garden surface'])

df

,Immoweb ID,property sub-type,Price,Building condition,Kitchen type,Bedrooms,Furnished,Tenement building,Number of frontages,Swimming pool,...,Post code_9970,Post code_9971,Post code_9981,Post code_9982,Post code_9988,Post code_9990,Post code_9991,Post code_9992,Terrace_Combined,garden_label
0,9729720,PENTHOUSE,179000.0,4.0,1.0,1.0,1,No,NaN,NaN,...,0,0,0,0,0,0,0,0,1.0,0
1,9729785,PENTHOUSE,255000.0,4.0,1.0,1.0,1,No,2.0,NaN,...,0,0,0,0,0,0,0,0,1.0,0
2,9729784,PENTHOUSE,255000.0,4.0,1.0,1.0,1,No,2.0,NaN,...,0,0,0,0,0,0,0,0,1.0,0
3,9729780,PENTHOUSE,620000.0,4.0,1.0,3.0,1,No,2.0,NaN,...,0,0,0,0,0,0,0,0,1.0,0
4,9727201,PENTHOUSE,379000.0,4.0,2.0,2.0,0,No,3.0,No,...,0,0,0,0,0,0,0,0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14536,7770552,SERVICE_FLAT,132000.0,6.0,0.0,1.0,0,No,NaN,NaN,...,0,0,0,0,0,0,0,0,0.0,0
14537,7770553,SERVICE_FLAT,177000.0,6.0,0.0,1.0,0,No,NaN,NaN,...,0,0,0,0,0,0,0,0,1.0,0
14538,7770551,SERVICE_FLAT,190000.0,6.0,0.0,2.0,0,No,NaN,NaN,...,0,0,0,0,0,0,0,0,1.0,0
14539,7921693,SERVICE_FLAT,1350000.0,2.0,0.0,30.0,0,No,NaN,NaN,...,0,0,0,0,0,0,0,0,0.0,0


### Garden orientation	


#### Drop garden orientation

In [33]:
df = df.drop(columns =['Garden orientation'])

df

,Immoweb ID,property sub-type,Price,Building condition,Kitchen type,Bedrooms,Furnished,Tenement building,Number of frontages,Swimming pool,...,Post code_9970,Post code_9971,Post code_9981,Post code_9982,Post code_9988,Post code_9990,Post code_9991,Post code_9992,Terrace_Combined,garden_label
0,9729720,PENTHOUSE,179000.0,4.0,1.0,1.0,1,No,NaN,NaN,...,0,0,0,0,0,0,0,0,1.0,0
1,9729785,PENTHOUSE,255000.0,4.0,1.0,1.0,1,No,2.0,NaN,...,0,0,0,0,0,0,0,0,1.0,0
2,9729784,PENTHOUSE,255000.0,4.0,1.0,1.0,1,No,2.0,NaN,...,0,0,0,0,0,0,0,0,1.0,0
3,9729780,PENTHOUSE,620000.0,4.0,1.0,3.0,1,No,2.0,NaN,...,0,0,0,0,0,0,0,0,1.0,0
4,9727201,PENTHOUSE,379000.0,4.0,2.0,2.0,0,No,3.0,No,...,0,0,0,0,0,0,0,0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14536,7770552,SERVICE_FLAT,132000.0,6.0,0.0,1.0,0,No,NaN,NaN,...,0,0,0,0,0,0,0,0,0.0,0
14537,7770553,SERVICE_FLAT,177000.0,6.0,0.0,1.0,0,No,NaN,NaN,...,0,0,0,0,0,0,0,0,1.0,0
14538,7770551,SERVICE_FLAT,190000.0,6.0,0.0,2.0,0,No,NaN,NaN,...,0,0,0,0,0,0,0,0,1.0,0
14539,7921693,SERVICE_FLAT,1350000.0,2.0,0.0,30.0,0,No,NaN,NaN,...,0,0,0,0,0,0,0,0,0.0,0


### How many fireplaces?	


#### Discard fireplaces

In [35]:
df = df.drop(columns =["How many fireplaces?"])

df

,Immoweb ID,property sub-type,Price,Building condition,Kitchen type,Bedrooms,Furnished,Tenement building,Number of frontages,Swimming pool,...,Post code_9970,Post code_9971,Post code_9981,Post code_9982,Post code_9988,Post code_9990,Post code_9991,Post code_9992,Terrace_Combined,garden_label
0,9729720,PENTHOUSE,179000.0,4.0,1.0,1.0,1,No,NaN,NaN,...,0,0,0,0,0,0,0,0,1.0,0
1,9729785,PENTHOUSE,255000.0,4.0,1.0,1.0,1,No,2.0,NaN,...,0,0,0,0,0,0,0,0,1.0,0
2,9729784,PENTHOUSE,255000.0,4.0,1.0,1.0,1,No,2.0,NaN,...,0,0,0,0,0,0,0,0,1.0,0
3,9729780,PENTHOUSE,620000.0,4.0,1.0,3.0,1,No,2.0,NaN,...,0,0,0,0,0,0,0,0,1.0,0
4,9727201,PENTHOUSE,379000.0,4.0,2.0,2.0,0,No,3.0,No,...,0,0,0,0,0,0,0,0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14536,7770552,SERVICE_FLAT,132000.0,6.0,0.0,1.0,0,No,NaN,NaN,...,0,0,0,0,0,0,0,0,0.0,0
14537,7770553,SERVICE_FLAT,177000.0,6.0,0.0,1.0,0,No,NaN,NaN,...,0,0,0,0,0,0,0,0,1.0,0
14538,7770551,SERVICE_FLAT,190000.0,6.0,0.0,2.0,0,No,NaN,NaN,...,0,0,0,0,0,0,0,0,1.0,0
14539,7921693,SERVICE_FLAT,1350000.0,2.0,0.0,30.0,0,No,NaN,NaN,...,0,0,0,0,0,0,0,0,0.0,0


### Surface of the plot




### Living area

In [40]:
df['Living area'].isnull().sum()

df['Living area'].describe()

count     9813.00000
mean       221.85346
std        297.35382
min         12.00000
25%         98.00000
50%        155.00000
75%        266.00000
max      11000.00000
Name: Living area, dtype: float64

### Price per square metre

In [37]:
df['price/m2'] = (df['Price']/ df['Living area']).round(2)

df

,Immoweb ID,property sub-type,Price,Building condition,Kitchen type,Bedrooms,Furnished,Tenement building,Number of frontages,Swimming pool,...,Post code_9971,Post code_9981,Post code_9982,Post code_9988,Post code_9990,Post code_9991,Post code_9992,Terrace_Combined,garden_label,price/m2
0,9729720,PENTHOUSE,179000.0,4.0,1.0,1.0,1,No,NaN,NaN,...,0,0,0,0,0,0,0,1.0,0,2841.27
1,9729785,PENTHOUSE,255000.0,4.0,1.0,1.0,1,No,2.0,NaN,...,0,0,0,0,0,0,0,1.0,0,6071.43
2,9729784,PENTHOUSE,255000.0,4.0,1.0,1.0,1,No,2.0,NaN,...,0,0,0,0,0,0,0,1.0,0,6071.43
3,9729780,PENTHOUSE,620000.0,4.0,1.0,3.0,1,No,2.0,NaN,...,0,0,0,0,0,0,0,1.0,0,4133.33
4,9727201,PENTHOUSE,379000.0,4.0,2.0,2.0,0,No,3.0,No,...,0,0,0,0,0,0,0,1.0,0,3158.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14536,7770552,SERVICE_FLAT,132000.0,6.0,0.0,1.0,0,No,NaN,NaN,...,0,0,0,0,0,0,0,0.0,0,NaN
14537,7770553,SERVICE_FLAT,177000.0,6.0,0.0,1.0,0,No,NaN,NaN,...,0,0,0,0,0,0,0,1.0,0,NaN
14538,7770551,SERVICE_FLAT,190000.0,6.0,0.0,2.0,0,No,NaN,NaN,...,0,0,0,0,0,0,0,1.0,0,NaN
14539,7921693,SERVICE_FLAT,1350000.0,2.0,0.0,30.0,0,No,NaN,NaN,...,0,0,0,0,0,0,0,0.0,0,900.00


### Post code

In [27]:
# df = pd.get_dummies(df_2, columns=['Post code'])


### Final check missing value %




In [28]:
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
missing_value_df

,column_name,percent_missing
Immoweb ID,Immoweb ID,0.0
property sub-type,property sub-type,0.0
Price,Price,0.0
Building condition,Building condition,0.0
Kitchen type,Kitchen type,0.0
...,...,...
Post code_9990,Post code_9990,0.0
Post code_9991,Post code_9991,0.0
Post code_9992,Post code_9992,0.0
Terrace_Combined,Terrace_Combined,0.0


# Data Analysis

### Group Post code to region

In [29]:
%pip install pgeocode --quiet

Note: you may need to restart the kernel to use updated packages.


In [30]:
import pgeocode
nomi = pgeocode.Nominatim("be")
nomi.query_postal_code("1040")['place_name']

'Etterbeek'

In [31]:
nomi.query_postal_code("5030")['state_name']

'Wallonie'

In [32]:
def get_state(col):  
    region = nomi.query_postal_code(col['Post code'])['state_name']
    if region == "Wallonie":
        region = "Wallonia"
    elif region == "Vlaanderen":
        region = "Flanders"
    else:
        region = "Brussels"
    return region

def get_commune(col):  
    return nomi.query_postal_code(col['Post code'])['place_name']

df['region'] = df.apply(lambda col: get_state(col), axis=1)
df['commune'] = df.apply(lambda col: get_commune(col), axis=1)


KeyError: 'Post code'

In [ ]:
df[['commune', 'Post code', 'region']]